##  moviepy로 비디오 처리하기

In [1]:
from moviepy.editor import VideoClip, VideoFileClip
from moviepy.editor import ipython_display
import cv2
import numpy as np
import os

### 비디오 읽고 쓰기

In [2]:
video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/video2.mp4'
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)
#ipython_display() 는 동영상을 주피터 노트북에 렌더링할 수 있게 도와주는 함수.
clip.ipython_display(fps=30, loop=True, autoplay=True, rd_kwargs=dict(logger=None))

result_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult.mp4'
clip.write_videofile(result_video_path)

t:   0%|          | 0/404 [00:00<?, ?it/s, now=None]                

Moviepy - Building video /home/aiffel0042/aiffel/video_sticker_app/images/mvpyresult.mp4.
MoviePy - Writing audio in mvpyresultTEMP_MPY_wvf_snd.mp3
MoviePy - Done.
Moviepy - Writing video /home/aiffel0042/aiffel/video_sticker_app/images/mvpyresult.mp4



Moviepy - Done !
Moviepy - video ready /home/aiffel0042/aiffel/video_sticker_app/images/mvpyresult.mp4


**moviepy** 를 이용해서 주피터 노트북 상에서 비디오를 읽고 쓰는 프로그램을 작성했습니다.

### 어둡게 만들기

In [3]:
# 위와 같이 동영상을 다시 읽어줍니다. 
video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/video2.mp4'
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)
clip.ipython_display(fps=30, loop=True, autoplay=True, rd_kwargs=dict(logger=None))

# clip에서 numpy로 데이터를 추출합니다. 
vlen=int(clip.duration*clip.fps)
video_container = np.zeros((vlen, clip.size[1], clip.size[0], 3), dtype=np.uint8)
for i in range(vlen):
    img = clip.get_frame(i/clip.fps)
    # 어둡게 만들기
    video_container[i] = (img * 0.5).astype(np.uint8)
    
# 새 clip 만들기 
dur = vlen / clip.fps
outclip = VideoClip(lambda t: video_container[int(round(t*clip.fps))], duration=dur)

# 쓰기
result_video_path2 = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult2.mp4'
outclip.write_videofile(result_video_path2, fps=30)

t:  14%|█▍        | 57/403 [00:00<00:00, 569.26it/s, now=None]

Moviepy - Building video /home/aiffel0042/aiffel/video_sticker_app/images/mvpyresult2.mp4.
Moviepy - Writing video /home/aiffel0042/aiffel/video_sticker_app/images/mvpyresult2.mp4



Moviepy - Done !
Moviepy - video ready /home/aiffel0042/aiffel/video_sticker_app/images/mvpyresult2.mp4


**moviepy** 로 읽은 동영상을 numpy 형태로 변환하고 영상 밝기를 50% 어둡게 만듭니다. frame 별로 이 과정이 진행됩니다. 

### 읽고 쓰는 시간을 측정하기

#### CASE 1 : moviepy 사용

In [4]:
start = cv2.getTickCount()
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)

vlen = int(clip.duration*clip.fps)
video_container = np.zeros((vlen, clip.size[1], clip.size[0], 3), dtype=np.uint8)

for i in range(vlen):
    img = clip.get_frame(i/clip.fps)
    video_container[i] = (img * 0.5).astype(np.uint8)

dur = vlen / clip.fps
outclip = VideoClip(lambda t: video_container[int(round(t*clip.fps))], duration=dur)

mvpy_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult.mp4'
outclip.write_videofile(mvpy_video_path, fps=30)

time = (cv2.getTickCount() - start) / cv2.getTickFrequency()
print (f'[INFO] moviepy time : {time:.2f}ms')

t:  14%|█▍        | 58/403 [00:00<00:00, 568.38it/s, now=None]

Moviepy - Building video /home/aiffel0042/aiffel/video_sticker_app/images/mvpyresult.mp4.
Moviepy - Writing video /home/aiffel0042/aiffel/video_sticker_app/images/mvpyresult.mp4



Moviepy - Done !
Moviepy - video ready /home/aiffel0042/aiffel/video_sticker_app/images/mvpyresult.mp4
[INFO] moviepy time : 2.54ms


#### CASE 2 : OpenCV 사용

In [6]:
start = cv2.getTickCount()
vc = cv2.VideoCapture(video_path)

cv_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/cvresult.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
vw = cv2.VideoWriter(cv_video_path, fourcc, 30, (640,360))

vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))

for i in range(vlen):
    ret, img = vc.read()
    if ret == False: break
    
    img_result = cv2.resize(img, (640, 360)) * 0.5
    vw.write(img_result.astype(np.uint8))
    
time = (cv2.getTickCount() - start) / cv2.getTickFrequency()
print (f'[INFO] cv time : {time:.2f}ms')

[INFO] cv time : 1.24ms


### moviepy 를 이용할 때의 장단점을 분석

moviepy을 이용하면 numpy를 사용할 수 있는 장점이 있습니다. 그래서 다양한 효과를 추가 할수 있습니다.  
하지만 openCV와 비교했을 때 moviepy은 2.54ms의 시간이 걸리고 CV는 1.24ms가 걸립니다.  
무려 두배나 차이나기 때문에 실시간 비디오나 대용량 비디오 처리에는 적합하지 않아 보입니다. 

## 어디까지 만들고 싶은지 정의하기

#### 1. 스티커앱 인식 거리 테스트

카페에서 실험한 결과 조명과 배경이 적절해서 그런지 인식이 잘 됐습니다.  
카메라에 얼굴이 잘릴 정도로 가까이 가면 인식이 잘 되지 않았습니다.  
팔보다 더 먼 거리에서 측정한 결과는 성능이 좋았습니다.  

#### 2. yaw, pitch, roll 각도 테스트

임의로 각도를 재는데 어려움이 있었습니다.  
그래서 사용자가 주로 촬영하는 각도에서 스티커가 잘 동작하는지를 지표로 삼았습니다.  
그 결과 yaw (고개를 좌우로 틀었을 때) 시야에서 카메라가 있을 때는 동작이 잘 됩니다.  
roll(고개를 갸우뚱 했을때) 45도 정도 밖으로 가면 인식이 잘 안됐습니다.  
pitch (고개를 끄덕였을 때) 앞선 각도보다 민감하게 작용했습니다.  

####  3. 스티커앱의 스펙 정하기

허용 거리: 셀카를 찍을 때 정도의 거리로 25cm ~ 1m  
인원 수: 4명 -> 한팔로 카메라를 들었을 때 카메라에 들어올 수 있는 인원수
허용 각도: pitch : -20 ~ 30도, yaw : -45 ~ 45도, roll : -45 ~ 45도- > 화면을 바라볼 수 있는 각도  
마스크 유무: 마스크를 써도 인식할 수 있는 스티커앱 만들기  

## 스티커 Out Bound 예외처리 하기

#### 1. 스티커앱의 예외 상황

스티커가 없는 경우 얼굴이 경계에서 벗어나도 예외 상황은 발생하지 않습니다.  
하지만 스티커가 좌우 경계 밖으로 나가는 경우 예외 상황이 발생하며 앱이 멈추게 됩니다.   

#### 2. 문제가 발생 코드 확인

newaddsticker.py의 img2sticker 메소드에서 왼쪽 경계를 벗어나서 detection 되는 경우 refined_x 의 값이 음수가 됩니다.  
그래서 예외 처리로 refined_x < 0일 때 스티커를 잘라주는 코드를 작성해줍니다. 

## 칼만 필터 적용하기

#### 1번 그림: 웹캠 결과, 2번 그림: 동영상 결과

<img src = "data/img/kf.png" width ="200" />   <img src = "data/img/kf1.png" width ="300" />

칼만 필터를 적용한 경우 얼굴을 detect 하지 못하는 문제가 발생했습니다.  
칼만 필터가 이전 측정값 전체의 평균을 사용하기 때문에 문제가 발생하는 줄 알고 수정을 하려고 했습니다.  
하지만 해당 코드에서는 이동 평균 필터를 사용하고 있었습니다.  
동영상으로 다시 테스트 한 결과 성능이 나아진 모습을 보였지만, 웹캠에 적용하는 데 문제가 있습니다. 